<a href="https://colab.research.google.com/github/Anjanisinghthakur/Google-Colab/blob/main/Pyspark_DF2_Handling_Nulls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

In [36]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Handling Nulls').master("local[*]").getOrCreate()

In [37]:
# handle nulls fillna()
schema_str="EMPLOYEE_ID int, FIRST_NAME string, LAST_NAME string, EMAIL string ,PHONE_NUMBER string, HIRE_DATE string,JOB_ID string, SALARY int,COMMISSION_PCT string, MANAGER_ID int, DEPARTMENT_ID int "
df=spark.read.csv("/content/employees.csv", header=True, schema=schema_str)

df=df.fillna({"Email":"Yet to provide","PHONE_NUMBER":"Not provided","JOB_ID": "Hiring in prog","MANAGER_ID":0 })
df.show()

+-----------+----------+---------+--------------+------------+---------+--------------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|         EMAIL|PHONE_NUMBER|HIRE_DATE|        JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------------+------------+---------+--------------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|      DOCONNEL|650.507.9833|21-Jun-07|      SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|        DGRANT|Not provided|13-Jan-08|      SH_CLERK|  2600|            - |         0|           50|
|        200|  Jennifer|   Whalen|       JWHALEN|515.123.4444|17-Sep-03|       AD_ASST|  4400|            - |       101|         NULL|
|        201|   Michael|Hartstein|      MHARTSTE|515.123.5555|17-Feb-04|        MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|Yet to provide|603.12

In [17]:
# handle nulls dropna()
schema_str="EMPLOYEE_ID int, FIRST_NAME string, LAST_NAME string, EMAIL string ,PHONE_NUMBER string, HIRE_DATE string,JOB_ID string, SALARY int,COMMISSION_PCT string, MANAGER_ID int, DEPARTMENT_ID int "
df=spark.read.csv("/content/employees.csv", header=True, schema=schema_str)
df=df.dropna(subset=["HIRE_DATE"])
df.show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-Jun-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|        NULL|13-Jan-08|  SH_CLERK|  2600|            - |      NULL|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-Sep-03|   AD_ASST|  4400|            - |       101|         NULL|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-Feb-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    NULL|603.123.6666|17-Aug-05|    MK_REP|  6000|            - |       201|           20|


In [35]:
# handle nulls using when &isNull()
from pyspark.sql.functions import when, coalesce, col
schema_str="EMPLOYEE_ID int, FIRST_NAME string, LAST_NAME string, EMAIL string ,PHONE_NUMBER string, HIRE_DATE string,JOB_ID string, SALARY int,COMMISSION_PCT string, MANAGER_ID int, DEPARTMENT_ID int "
df=spark.read.csv("/content/employees.csv", header=True, schema=schema_str)
df=df.withColumn("new_dept", when(df.DEPARTMENT_ID.isNull(),0).otherwise(df.DEPARTMENT_ID))
df.show()
# using coalesce
df=df.withColumn("new_manager", coalesce(df.MANAGER_ID,df.DEPARTMENT_ID))
df.show()
df.printSchema()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+--------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|new_dept|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+--------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-Jun-07|  SH_CLERK|  2600|            - |       124|           50|      50|
|        199|   Douglas|    Grant|  DGRANT|        NULL|13-Jan-08|  SH_CLERK|  2600|            - |      NULL|           50|      50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-Sep-03|   AD_ASST|  4400|            - |       101|         NULL|       0|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-Feb-04|    MK_MAN| 13000|            - |       100|           20|      20|
|        202|       Pat|      Fay|    NULL|603.123.6666|17-Aug